In [12]:
import sys
sys.path.append("../src/")
from sgpi.util import get_dataset, get_xy, get_r_squared, split
from sgpi import model
from sgpi.learner import SGRegressionLearner
import sqlalchemy as sa
from sqlalchemy import orm, func

In [2]:
ses = model.make_session()

In [3]:
best_results = ses.query(model.Result, func.min(model.Result.validation_mse)).group_by(model.Result.experiment_id).all()

In [8]:
def retrain(res):
    df = get_dataset(res.experiment.dataset)
    train, test = split(df)
    X_train, y_train = get_xy(train)
    X_test, y_test = get_xy(test)
    estimator = SGRegressionLearner(res.grid_config, res.regularization_config, res.solver_config, res.final_solver_config, res.adaptivity_config)
    estimator.fit(X_train, y_train)
    score_train = -estimator.score(X_train, y_train)
    score_test = -estimator.score(X_test, y_test)
    score_train, score_test
    res.train_grid_points = estimator.get_grid_size()
    res.train_mse = score_train
    res.train_r2 = get_r_squared(estimator, X_train, y_train)
    res.test_mse = score_test
    res.test_r2 = get_r_squared(estimator, X_test, y_test)

In [9]:
for res in best_results:
    retrain(res[0])

In [11]:
ses.commit()

In [13]:
from sgpi.grid_search import GridSearch
GridSearch()

TypeError: __init__() takes exactly 4 arguments (1 given)